In [149]:
import numpy as np
import math
import pandas as pd

In [150]:
def return_unique_characteristics(column: int, datos: list) -> list:
    characteristics = datos[:,column].tolist()
    characteristics

    unique_characteristics = []

    for characteristic in characteristics:
        if not(characteristic in unique_characteristics):
            unique_characteristics.append(characteristic)
    return unique_characteristics

def create_dictionary(unique_characteristics: list, total_class: int) -> dict:
    dictionary = dict()

    for characteristic in unique_characteristics:    
        for i in np.arange(total_class):
            key = str(characteristic) + str(i)
            dictionary.setdefault(key, 0)
    return dictionary

def gini(total_class: int, column: int, column_class: int, datos: list):
    unique_characteristics = return_unique_characteristics(column=column)
    dictionary = create_dictionary(unique_characteristics=unique_characteristics, total_class=total_class)

    for i in np.arange(datos.shape[0]):
        key = datos[i,column] + datos[i, column_class]
        dictionary[key] = dictionary[key] + 1


    total_sum = 0
    for characteristic in unique_characteristics:
        simple_sum = 0
        ni = 0
        for i in np.arange(total_class):
            key = str(characteristic) + str(i)
            simple_sum += dictionary[key] / datos.shape[0]
            ni += dictionary[key]
        simple_sum = 1 - simple_sum
        total_sum += (simple_sum * ni) / datos.shape[0] 
    print(total_sum)

def entropia(total_class: int, column: int, column_class: int, datos: list):
    unique_characteristics = return_unique_characteristics(column=column, datos=datos)
    dictionary = create_dictionary(unique_characteristics=unique_characteristics, total_class=total_class)

    for i in np.arange(datos.shape[0]):
        key = str(datos[i,column]) + str(datos[i, column_class])
        dictionary[key] = dictionary[key] + 1


    total_sum = 0
    for characteristic in unique_characteristics:
        simple_sum = 0
        ni = 0
        for i in np.arange(total_class):
            key = str(characteristic) + str(i)
            res = (dictionary[key] / datos.shape[0])
            if(res):
                res = res * math.log2(dictionary[key] / datos.shape[0])
            simple_sum += res
            ni += dictionary[key]
        simple_sum = - simple_sum
        total_sum += (simple_sum * ni) / datos.shape[0] 
    return total_sum

import pandas as pd
import scipy.stats as stats

def correlacion_Pearson(df, result):
    columna_1 = df.iloc[:, result[0][0]]

    # Calcula la asociación entre la primera columna y el resto de columnas utilizando la prueba de chi-cuadrado
    asociaciones = []
    for columna in df.columns[:-1]:  # Excluimos la última columna (clase)
        tabla_contingencia = pd.crosstab(columna_1, df[columna])
        _, p_value, _, _ = stats.chi2_contingency(tabla_contingencia)
        asociaciones.append(p_value)

    return asociaciones


# **Twitter**

In [151]:
df = pd.read_csv('Twitter.csv')

In [152]:
df.head()

,Unnamed: 0,nombre,cantidad_caracteres_nombre,tiene_foto_perfil,cantidad_seguidores,cantidad_seguidos,promedio_publicaciones,perfil_privado,dia_mas_publica,comenta_publicaciones,clase
0,0,Catalin31937081,largo,Sí,bajo,bajo,bajo,No,Sábado,Sí,Fake
1,1,JadeCrespo8,medio,Sí,bajo,medio,bajo,No,Martes,Sí,Fake
2,2,V12Ara,corto,Sí,bajo,bajo,bajo,No,Martes,Sí,Fake
3,3,BriannaLun_,medio,Sí,bajo,bajo,bajo,No,Martes,Sí,Fake
4,4,IsoldaMancillas,largo,Sí,bajo,bajo,bajo,No,Martes,Sí,Fake


In [153]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^nombre')]

df['clase'] = df['clase'].replace({'Fake': 0, 'Real': 1})
df['clase'] = df['clase'].replace({'FALSO': 0, 'real': 1})

In [154]:
df.tail()

,cantidad_caracteres_nombre,tiene_foto_perfil,cantidad_seguidores,cantidad_seguidos,promedio_publicaciones,perfil_privado,dia_mas_publica,comenta_publicaciones,clase
289,medio,si,bajo,bajo,bajo,no,sabado,no,0
290,medio,si,bajo,bajo,bajo,no,domingo,no,0
291,largo,si,bajo,bajo,bajo,no,miercoles,no,0
292,largo,si,medio,alto,bajo,no,sabado,no,0
293,medio,si,medio,medio,bajo,no,miercoles,no,0


In [155]:
twitter = df.to_numpy()

entropis = []
for i in np.arange(8):
    total_class = 2
    column_class = 8
    entropis.append(entropia(total_class=total_class, column=i, column_class=column_class, datos=twitter))
entropis = np.array(entropis)
print(entropis)

[0.85632273 0.83023047 0.7598031  0.78977323 0.8717221  0.82573526
 0.33491488 0.72101021]


In [156]:
print(entropis.min())

0.33491488013992987


In [157]:
result = np.where(entropis == entropis.min())

column_names = df.columns.tolist()
print(column_names[result[0][0]])

dia_mas_publica


In [158]:
asociaciones = correlacion_Pearson(df=df, result=result)
print(asociaciones)

[0.41557326294720204, 4.0343519405537154e-48, 0.0016279983266533596, 0.006501242160420438, 0.5245844962650021, 2.942462847231813e-50, 0.0, 1.069340807379657e-43]


In [159]:
copy_asociaciones = asociaciones.copy()
copy_asociaciones.sort()

result1 = np.where(asociaciones == copy_asociaciones[1])
result2 = np.where(asociaciones == copy_asociaciones[2])

print(column_names[result1[0][0]])
print(column_names[result2[0][0]])

perfil_privado
tiene_foto_perfil


# **Facebook**

In [160]:
df = pd.read_csv('Facebook.csv')

In [161]:
df.head()

,Unnamed: 0,caracteres_nombre,foto_perfil,num_amigos,dias_mas_Publicaciones,paginas_que_sigue,Clase
0,0,corto,Sí,medio,Domingo,pocas,T
1,1,medio,Sí,medio,Domingo,pocas,T
2,2,medio,Sí,medio,Domingo,pocas,T
3,3,corto,Sí,medio,Domingo,pocas,T
4,4,corto,Sí,medio,Viernes,pocas,T


In [162]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Clase'] = df['Clase'].replace({'T': 0, 'F': 1})

In [163]:
df.tail()

,caracteres_nombre,foto_perfil,num_amigos,dias_mas_Publicaciones,paginas_que_sigue,Clase
356,largo,SI,medio,Sabado,algunas,0
357,medio,SI,bajo,ninguno,pocas,1
358,corto,SI,bajo,ninguno,pocas,1
359,medio,SI,alto,ninguno,pocas,1
360,largo,SI,medio,ninguno,pocas,0


In [164]:
facebook = df.to_numpy()

entropis = []
for i in np.arange(4):
    total_class = 2
    column_class = 5
    entropis.append(entropia(total_class=total_class, column=i, column_class=column_class, datos=facebook))
entropis = np.array(entropis)
print(entropis)

[0.83703369 0.80202375 0.75917363 0.60933213]


In [165]:
print(entropis.min())

0.6093321268960614


In [166]:
result = np.where(entropis == entropis.min())

column_names = df.columns.tolist()
print(column_names[result[0][0]])

dias_mas_Publicaciones


In [167]:
asociaciones = correlacion_Pearson(df=df, result=result)
print(asociaciones)

[0.3591110033562538, 2.0159185732948584e-37, 1.896428313198007e-06, 0.0, 0.09272798349361645]


In [168]:
copy_asociaciones = asociaciones.copy()
copy_asociaciones.sort()

result1 = np.where(asociaciones == copy_asociaciones[1])
result2 = np.where(asociaciones == copy_asociaciones[2])


print(column_names[result1[0][0]])
print(column_names[result2[0][0]])

foto_perfil
num_amigos


# **Reconocimiento Perfiles**

In [169]:
df = pd.read_csv('ReconocimientoPerfiles.csv')

In [170]:
df.head()

,Unnamed: 0,Edad,Genero,Region,Integrantes-Familia-num,Proveedor-Familia,Tutor,Vive-con-Familia,Enfermedad-CrÃ³nica,Carrera,...,Noviazgo,Actividad-Cultural-o-Deportiva,Tiempo-con-sus-amigos-compaÃ±eros,Horas-Semana-Divertirse con-sus-amigos,ConsumÃ­a-Alcohol-Entre-semana,ConsumÃ­a-Alcohol-Fin-semana,Horas-DÃ­a-Dedicada-Redes-Sociales,Gustan-Videojuegos,Horas-Semana-Dedicada-Videojuegos,Horas-Semana-Dedicada-Actividades-Entretenimiento
0,0,señor,Masculino,Sierra Sur,muchos,Ambos,Madre,No,No,Ingeniería en Computación,...,No,No,No,baja,nunca,nunca,<1,No,0,1-3
1,1,señor,Masculino,Valles Centrales,algunos,Mamá,Madre,Sí,No,Ingeniería en Computación,...,Sí,Sí,Sí,media,casi-nunca,a veces,>2,Sí,>3,1-3
2,2,señor,Masculino,Otro estado,algunos,Papá,Padre,Sí,No,Ingeniería en Computación,...,Sí,No,Sí,baja,casi-nunca,casi-nunca,1-2,Sí,1-3,1-3
3,3,joven,Masculino,Mixteca,muchos,Mamá,Madre,No,No,Ingeniería en Computación,...,No,Sí,Sí,baja,a veces,regularmente,1-2,No,0,1-3
4,4,joven,Masculino,Mixteca,algunos,Papá,Madre,Sí,No,Ingeniería en Computación,...,Sí,No,No,baja,nunca,casi-nunca,1-2,Sí,>3,>3


In [171]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df['Promedio-Primer-Semestre'] = df['Promedio-Primer-Semestre'].replace({'malo': 0, 'regular': 1, 'bueno': 2})

df = df.loc[:, ['Edad','Genero','Region','Integrantes-Familia-num','Proveedor-Familia','Tutor','Vive-con-Familia','Enfermedad-CrÃ³nica','Carrera','Semestre','Preparatoria','Promedio-Preparatoria','Materias-Relacionadas','Carrera-Elegida-Ajusta-A-Gustos-Habilidades','Buena-RelaciÃ³n-Familiar','Buena-RelaciÃ³n-Amigos-CompaÃ±eros','Materias-Cursadas-Primer-Semestre','Materias-Aprobadas-Primer-Semestre','Materias-Reprobadas-De-ComputaciÃ³n','Materias-Reprobadas-De-MatemÃ¡ticas','Materias-Reprobadas-De-Ciensias-Sociales','Horas-Promedio-Estudio-Examenes','Horas-Promedio-Estudio-Actividades-Escolares','Cursos-Extra-Primer-Semestre','Noviazgo','Actividad-Cultural-o-Deportiva','Tiempo-con-sus-amigos-compaÃ±eros','Horas-Semana-Divertirse con-sus-amigos','ConsumÃ­a-Alcohol-Entre-semana','ConsumÃ­a-Alcohol-Fin-semana','Horas-DÃ­a-Dedicada-Redes-Sociales','Gustan-Videojuegos','Horas-Semana-Dedicada-Videojuegos','Horas-Semana-Dedicada-Actividades-Entretenimiento','Promedio-Primer-Semestre']]

In [172]:
df = df.drop(['Region', 'Integrantes-Familia-num','Proveedor-Familia','Carrera','Semestre','Preparatoria'], axis=1)

In [173]:
df.tail()

,Edad,Genero,Tutor,Vive-con-Familia,Enfermedad-CrÃ³nica,Promedio-Preparatoria,Materias-Relacionadas,Carrera-Elegida-Ajusta-A-Gustos-Habilidades,Buena-RelaciÃ³n-Familiar,Buena-RelaciÃ³n-Amigos-CompaÃ±eros,...,Actividad-Cultural-o-Deportiva,Tiempo-con-sus-amigos-compaÃ±eros,Horas-Semana-Divertirse con-sus-amigos,ConsumÃ­a-Alcohol-Entre-semana,ConsumÃ­a-Alcohol-Fin-semana,Horas-DÃ­a-Dedicada-Redes-Sociales,Gustan-Videojuegos,Horas-Semana-Dedicada-Videojuegos,Horas-Semana-Dedicada-Actividades-Entretenimiento,Promedio-Primer-Semestre
137,joven,Masculino,Madre,No,No,bueno,Sí,No,Sí,Sí,...,No,No,baja,nunca,nunca,<1,Sí,1-3,1-3,0
138,joven,Femenino,Madre,No,No,bueno,No,No,Sí,No,...,No,Sí,media,nunca,casi-nunca,>2,No,0,>3,0
139,señor,Masculino,Pariente,No,No,regular,No,No,No,No,...,No,No,baja,nunca,nunca,0,No,0,1-3,0
140,joven,Masculino,Madre,No,No,bueno,Sí,Sí,Sí,Sí,...,Sí,Sí,baja,nunca,nunca,>2,Sí,1-3,1-3,1
141,joven,Masculino,Madre,Sí,No,bueno,Sí,No,Sí,Sí,...,No,No,baja,nunca,nunca,>2,Sí,>3,>3,0


In [189]:
perfiles = df.to_numpy()

entropis = []
for i in np.arange(29):
    total_class = 3
    column_class = 28
    entropis.append(entropia(total_class=total_class, column=i, column_class=column_class, datos=perfiles))
entropis = np.array(entropis)
entropis = np.delete(entropis, -1)
print(entropis)

[1.09293454 1.10494314 1.00427021 1.08446664 1.16507837 1.08792193
 1.07304613 1.07998783 1.13797752 1.08811103 1.08563791 1.00427979
 1.00336711 1.02930728 1.15124587 1.07195744 1.1021616  1.12711205
 1.08999735 1.10133315 1.09851236 0.97555931 1.01693176 0.93909547
 0.9409765  1.09120182 0.96358662 1.05634074]


In [175]:
print(entropis.min())

0.9390954682061301


In [223]:
result = np.where(entropis == entropis.min())

column_names = df.columns.tolist()
print(column_names[result[0][0]])

ConsumÃ­a-Alcohol-Fin-semana


In [224]:
asociaciones = correlacion_Pearson(df=df, result=result)
print(asociaciones)

[0.6210990645550556, 0.5714763133193876, 0.871164109725793, 0.14096176702519048, 0.6081410036835326, 0.6157624274362139, 0.7337761557328277, 0.8376671779116189, 0.328141487648242, 0.2267876809241921, 0.22536720404787783, 0.3462826275337519, 0.259980503807206, 0.5265069813375043, 0.7775019869102127, 0.7144102989059651, 0.24313297527576005, 0.44921113960338777, 0.5130232311442953, 0.5545305459227283, 0.037194046944004334, 0.008589399887949316, 3.5869459153454846e-21, 3.855857630673099e-86, 0.8912965179667474, 0.41948573801658295, 0.35734058569313454, 0.6385006181308563]


In [225]:
copy_asociaciones = asociaciones.copy()
copy_asociaciones.sort()

result1 = np.where(asociaciones == copy_asociaciones[1])
result2 = np.where(asociaciones == copy_asociaciones[2])

print(column_names[result1[0][0]])
print(column_names[result2[0][0]])

ConsumÃ­a-Alcohol-Entre-semana
Horas-Semana-Divertirse con-sus-amigos
